# console capture and replay for notebooks

This notebook shows an example of an small utility that can help capture the output sent to stdout by a long-running process in a notebook.

In [1]:
from __future__ import print_function

from dl_helper import console_capture as cc

A preliminary check: this cell should print

  `789...`

since the carriage return should make it overwrite all previous output. If not, this is possibly a [buggy notebook version](https://github.com/jupyter/notebook/issues/1970). Update notebook to at least 4.3.1


In [2]:
a = '123...\r456...\r789...'
print(a)

789...


Backspaces are not correctly processed by notebooks, though (they are ignored). So this one should print

   `abc...`

but it will not:

In [3]:
a = '123...\r456...\r789\b\b\babc...'
print(a)

789abc...


# High-level API

## Process example

To use the high-level API we need to express our processing task in the form of a [callable](https://docs.python.org/3/library/functions.html#callable)  (a function, method, or in general any object with a `__call__` [attribute](https://docs.python.org/3/reference/datamodel.html#object.__call__)). The callable can take arbitrary arguments (as we need to parametrize the processing or send options).

In [4]:
import time

def processing_call( num, msg, **kwargs ):
    '''
    A function that basically does nothing, but takes its time to do 
    that nothing and prints out progress reports while it is doing it
      :param num: how many empty iterations to do
      :param msg: Message to be printed at processing end
      :param kwargs: Additional unused arguments, also to be printed at processing end
    '''
    start = time.time()
    # Outer loop: 'num' times
    for j in range(num):
        # Inner loop: 10 times
        for i in range(10):
            # Print progress reports. Use control characters to overwrite the line
            print( "\b\b\b\b\b\b\b\b\b\b\b\r", '{:2} {:2} {:5.2f}'.format( j+1, i+1, time.time()-start ), end=' ' )
            time.sleep(0.10)
        print('<iteration done>') # next line
        time.sleep(1)
    print( "Done! Msg={} Param={}".format(msg,kwargs) )

## Basic style
The basic procedure is creating a wrap object by passing it the callable with its required arguments ...

In [5]:
p = cc.ProcessWrap( processing_call, 4, "Hi", param="I'm a process" )

... and then calling the `process()` method on that object. It appears to be the same as executing the callable directly: the process is carried out, the cell appears as **`[*]`** (i.e. working) and the notebook as _busy_ and the progress results printed by the callable are written back to the notebook.

In [6]:
p.process()

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.93 <iteration done>
  3 10  4.95 <iteration done>
  4 10  6.97 <iteration done>
Done! Msg=Hi Param={'param': "I'm a process"}


But behind the scenes, the printed data has been written to a temporal file, whose contents are automatically loaded upon termination as an internal data object (and the file is also automatically deleted), so they can be reprinted at will:

In [7]:
p.show()

----- DONE -----
  1 10  0.91 <iteration done>
  2 10  2.93 <iteration done>
  3 10  4.95 <iteration done>
  4 10  6.97 <iteration done>
Done! Msg=Hi Param={'param': "I'm a process"}


### accessing raw data

The same output is also available in raw form through the `data` attribute of the object:

In [8]:
p.data

'\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  1  0.00 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  2  0.10 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  3  0.20 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  4  0.31 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  5  0.41 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  6  0.51 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  7  0.61 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  8  0.71 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1  9  0.81 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  1 10  0.91 <iteration done>\n\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  1  2.02 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  2  2.12 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  3  2.22 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  4  2.32 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  5  2.42 \x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\r  2  6  2.52 \x08\x08\x08\x08\x08\x08\x08\x08\x08\

Obviously all those control characters make the data a bit difficult to read. We can use the provided `clean_string()` function, which applies backspaces (`\b`) and carriage returns (`\r`) as they would be processed were the data sent to standard output:

In [9]:
cc.clean_string( p.data )

'  1 10  0.91 <iteration done>\n  2 10  2.93 <iteration done>\n  3 10  4.95 <iteration done>\n  4 10  6.97 <iteration done>\nDone! Msg=Hi Param={\'param\': "I\'m a process"}\n'

### closing and reopening the notebook window

More importantly, the `process()` call, though it seems to be blocking it is doing so by actually looping over a check-sleep cycle. The actual process is being carried out in a **background thread**, which will continue saving its results to the temporal file even if 
* the browser window is closed 
* or the kernel is interrupted (since that intrruption will stop only the main thread, not the background thread)

Opening the notebook later on and executing the `show()` method will print out the results stored in the file. Moreover, it will reconnect the standard output of the process with the notebook cell, so that further output gets back to the notebook. We can test it with a longer running process:

In [10]:
p2 = cc.ProcessWrap( processing_call, 50, "Hi" )

In [ ]:
p2.process()  
# after launching execution of this cell, save the notebook & close the browser window 
# (confirm the warning about leaving the page)
# note: close the window, NOT the notebook

Launching process ... 
  1 10  0.92 <iteration done>
  2 10  2.94 <iteration done>


In [12]:
# Then come back again here, INTERRUPT the kernel and execute this cell. 
# It should print all output received so far, and continue from there
p2.show() 

----- RUNNING -----
  1 10  0.92 <iteration done>
  2 10  2.94 <iteration done>
  3 10  4.96 <iteration done>
  4 10  6.99 <iteration done>
  5 10  9.02 <iteration done>
  6 10 11.04 <iteration done>
  7 10 13.05 <iteration done>
  8 10 15.07 <iteration done>
  9 10 17.08 <iteration done>
 10 10 19.09 <iteration done>
 11 10 21.12 <iteration done>
 12 10 23.14 <iteration done>
 13 10 25.17 <iteration done>
 14 10 27.19 <iteration done>
 15 10 29.20 <iteration done>
 16 10 31.23 <iteration done>
 17 10 33.24 <iteration done>
 18 10 35.27 <iteration done>
 19 10 37.28 <iteration done>
 20 10 39.30 <iteration done>
 21 10 41.32 <iteration done>
 22 10 43.33 <iteration done>
 23 10 45.36 <iteration done>
 24 10 47.37 <iteration done>
 25 10 49.38 <iteration done>
 26 10 51.40 <iteration done>
 27 10 53.43 <iteration done>
 28 10 55.45 <iteration done>
 29 10 57.47 <iteration done>
 30 10 59.48 <iteration done>
 31 10 61.51 <iteration done>
 32 10 63.53 <iteration done>
 33 10 65.54 <iterat

 49 10 97.85 <iteration done>
 50 10 99.87 <iteration done>
Done! Msg=Hi Param={}


There is one caveat: as seen in the previous cell, the `process()` call was blocking the notebook (by doing the mentioned check-sleep cycle), so coming back to the notebook (opening its window again) will make it appear unresponsive. 

You will need to **interrupt** the kernel to stop that `process()` cell so that it is possible to execute a `show()` cell. This is not as disruptive as it seems, since what it is being interrupted is *the foreground process*, not the background thread that it is doing the real processing. But nevertheless if it is known for sure that the window will be closed and reopened, it will be better to use the non-blocking call below.

## Non-blocking usage

This modality works the same way, but adding the `block=False` parameter to the `process()` method: 

In [13]:
pn = cc.ProcessWrap( processing_call, 50, "Look! NonBlocking!" )

In [14]:
pn.process( block=False )
# Now the notebook appears free to use (not busy), though the background thread 
# is still pushing text to the cell output
# We can save & close the notebook here. There will still be a "confirm" warning, probably.

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.93 <iteration done>
  3 10  4.95 <iteration done>
  4 10  6.98 4  9  6.87<iteration done>
  5 10  8.99 <iteration done>
  6 10 11.01 <iteration done>
  7 10 13.03 <iteration done>
  8  4 14.43 

In [15]:
# After coming back to the notebook, in this modality it is no longer necessary to interrupt the kernel. 
# Just execute this cell to reconnect with the processing cell output
pn.show()

----- RUNNING -----
  1 10  0.91 <iteration done>
  2 10  2.93 <iteration done>
  3 10  4.95 <iteration done>
  4 10  6.98 <iteration done>
  5 10  8.99 <iteration done>
  6 10 11.01 <iteration done>
  7 10 13.03 <iteration done>
  8 10 15.04 <iteration done>
  9 10 17.05 <iteration done>
 10 10 19.06 <iteration done>
 11 10 21.09 <iteration done>
 12 10 23.11 <iteration done>
 13 10 25.13 <iteration done>
 14 10 27.16 <iteration done>
 15 10 29.18 <iteration done>
 16 10 31.20 <iteration done>
 17 10 33.22 <iteration done>
 18 10 35.24 <iteration done>
 19 10 37.26 <iteration done>
 20 10 39.27 <iteration done>
 21 10 41.30 <iteration done>
 22 10 43.32 <iteration done>
 23 10 45.33 <iteration done>
 24 10 47.35 <iteration done>
 25 10 49.37 <iteration done>
 26 10 51.39 <iteration done>
 27 10 53.41 <iteration done>
 28 10 55.43 <iteration done>
 29 10 57.46 <iteration done>
 30 10 59.47 <iteration done>
 31 10 61.51 <iteration done>
 32 10 63.53 <iteration done>


 33 10 65.55 <iteration done>
 34 10 67.57 <iteration done>
 35 10 69.58 <iteration done>
 36 10 71.61 <iteration done>
 37 10 73.63 <iteration done>
 38 10 75.65 <iteration done>
 39 10 77.67 <iteration done>
 40 10 79.69 <iteration done>
 41 10 81.71 <iteration done>
 42 10 83.73 <iteration done>
 43 10 85.75 <iteration done>
 44 10 87.77 <iteration done>
 45 10 89.79 <iteration done>
 46 10 91.82 <iteration done>
 47 10 93.84 <iteration done>
 48 10 95.85 8  3 95<iteration done>
 49 10 97.88 <iteration done>
 50 10 99.90 <iteration done>
Done! Msg=Look! NonBlocking! Param={}


In [16]:
# We can still print the collected output, after it has finished
pn.show()

----- DONE -----
  1 10  0.91 <iteration done>
  2 10  2.93 <iteration done>
  3 10  4.95 <iteration done>
  4 10  6.98 <iteration done>
  5 10  8.99 <iteration done>
  6 10 11.01 <iteration done>
  7 10 13.03 <iteration done>
  8 10 15.04 <iteration done>
  9 10 17.05 <iteration done>
 10 10 19.06 <iteration done>
 11 10 21.09 <iteration done>
 12 10 23.11 <iteration done>
 13 10 25.13 <iteration done>
 14 10 27.16 <iteration done>
 15 10 29.18 <iteration done>
 16 10 31.20 <iteration done>
 17 10 33.22 <iteration done>
 18 10 35.24 <iteration done>
 19 10 37.26 <iteration done>
 20 10 39.27 <iteration done>
 21 10 41.30 <iteration done>
 22 10 43.32 <iteration done>
 23 10 45.33 <iteration done>
 24 10 47.35 <iteration done>
 25 10 49.37 <iteration done>
 26 10 51.39 <iteration done>
 27 10 53.41 <iteration done>
 28 10 55.43 <iteration done>
 29 10 57.46 <iteration done>
 30 10 59.47 <iteration done>
 31 10 61.51 <iteration done>
 32 10 63.53 <iteration done>
 33 10 65.55 <iteration

## Additional options

### verbose

Adding `verbose=False` to the `process()` method will suppress output (but still save to the temporal file)

In [17]:
pq = cc.ProcessWrap( processing_call, 3, "Now Quiet!" )
pq.process( verbose=False )
# This will take a few seconds to run, with no output produced while it is running

In [18]:
# After finishing, we can now print the captured data
pq.show()

----- DONE -----
  1 10  0.90 <iteration done>
  2 10  2.91 <iteration done>
  3 10  4.91 <iteration done>
Done! Msg=Now Quiet! Param={}


### delete

The temporal file is automatically deleted upon task termination. This is usually not a problem because the data in the file is loaded onto the `ProcessWrap` object, and it can be printed via `show()` (or retrieved through the `data` attribute). But of course it will be lost if the notebook kernel is terminated (_close and halt_). If the processing is very long or important, as a safety measure the `delete=False` parameter to `process()` will cancel file removal.

In [19]:
pk = cc.ProcessWrap( processing_call, 3, "Not deleted!" )
pk.process( delete=False )

Launching process ... 
  1 10  0.91 <iteration done>
  2 10  2.93 <iteration done>
  3 10  4.95 <iteration done>
Done! Msg=Not deleted! Param={}


In [20]:
# See the filename that was used
pk.logname

'/vm/Vagrant/machine-learning-vm/notebook/vmfiles/IPNB/Misc/notebook-d6yhvh15.log'

In [21]:
# The data is available for posterity in the logfile
# We use the io.open 'newline' argument so that (in Python 3) we avoid newline translation
import io
with io.open(pk.logname, "r", newline='') as f:
    print( f.read() )

  1 10  0.91 <iteration done>
  2 10  2.93 <iteration done>
  3 10  4.95 <iteration done>
Done! Msg=Not deleted! Param={}



In [22]:
# We can still delete it manually, of course
import os
os.unlink( pk.logname )

# Low level API
The `ProcessWrap` object functionality is implemented via lower level objects:

* `OutputDest` is a file-like object that substitutes _stdout_ & _stderr_ to perform the data capture
* `ConsoleCapture` provides an object that uses `OutputDest`, and `ConsoleCaptureCtx` a context manager for it
* `ProcessingThread` provides the processing thread that encapsulates its output via `ConsoleCaptureCtx`

These objects can be used directly if more specialised treatment is needed

### ConsoleCaptureCtx


In [23]:
import datetime

with cc.ConsoleCaptureCtx(verbose=False) as ll1:
    for i in range(15):
        print( '{:3}  {}'.format(i+1, datetime.datetime.now()) )

In [24]:
ll1.reprint();

  1  2019-05-08 11:58:02.467537
  2  2019-05-08 11:58:02.467596
  3  2019-05-08 11:58:02.467617
  4  2019-05-08 11:58:02.467638
  5  2019-05-08 11:58:02.467656
  6  2019-05-08 11:58:02.467672
  7  2019-05-08 11:58:02.467693
  8  2019-05-08 11:58:02.467709
  9  2019-05-08 11:58:02.467725
 10  2019-05-08 11:58:02.467740
 11  2019-05-08 11:58:02.467756
 12  2019-05-08 11:58:02.467771
 13  2019-05-08 11:58:02.467790
 14  2019-05-08 11:58:02.467807
 15  2019-05-08 11:58:02.467822


In [25]:
ll1.data

'  1  2019-05-08 11:58:02.467537\n  2  2019-05-08 11:58:02.467596\n  3  2019-05-08 11:58:02.467617\n  4  2019-05-08 11:58:02.467638\n  5  2019-05-08 11:58:02.467656\n  6  2019-05-08 11:58:02.467672\n  7  2019-05-08 11:58:02.467693\n  8  2019-05-08 11:58:02.467709\n  9  2019-05-08 11:58:02.467725\n 10  2019-05-08 11:58:02.467740\n 11  2019-05-08 11:58:02.467756\n 12  2019-05-08 11:58:02.467771\n 13  2019-05-08 11:58:02.467790\n 14  2019-05-08 11:58:02.467807\n 15  2019-05-08 11:58:02.467822\n'

### ConsoleCapture

In [26]:
ll2 = cc.ConsoleCapture()

In [27]:
# Print something (that will not be captured)
print( "A")
print( "B" )

A
B


In [28]:
# Start capturing, write something and stop capturing
ll2.start()

print( "CDE...\r", end='')
print( "...FGH")
print( "IJK")

ll2.stop();

...FGH
IJK


In [29]:
# Print what was captured, and remove the logfile
ll2.reprint().remove();

...FGH
IJK
